In [1]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple
from itertools import chain, repeat
from pprint import pformat, pprint
from typing import (
    AbstractSet, Any, Callable, Collection, Dict, Generator, Generic, Iterable, List, Mapping,
    NamedTuple, Optional, Tuple, TypeVar, Union
)

from graphql import GraphQLList, GraphQLString, parse
from graphql.utils.build_ast_schema import build_ast_schema
from graphql_compiler.compiler.compiler_frontend import graphql_to_ir
from graphql_compiler.compiler.blocks import (
    Backtrack, CoerceType, ConstructResult, EndOptional, Filter, GlobalOperationsStart, 
    MarkLocation, Recurse, QueryRoot, Traverse
)
from graphql_compiler.compiler.compiler_entities import BasicBlock, Expression
from graphql_compiler.compiler.expressions import (
    BinaryComposition, ContextField, ContextFieldExistence, Literal, LocalField, 
    OutputContextField, TernaryConditional, Variable
)
from graphql_compiler.compiler.compiler_frontend import IrAndMetadata, graphql_to_ir
from graphql_compiler.compiler.helpers import Location, get_only_element_from_collection
from graphql_compiler.schema import GraphQLDate, GraphQLDateTime, GraphQLDecimal
from graphql_compiler.tests.test_helpers import SCHEMA_TEXT

In [2]:
def make_empty_stack():
    return ImmutableStack(None, 0, None)
    

class ImmutableStack(NamedTuple):
    value: Any
    depth: int
    tail: Optional['ImmutableStack']
        
    def peek(self) -> Any:
        return self.value
    
    def push(self, value: Any):
        return ImmutableStack(value, self.depth + 1, self)
    
    def pop(self) -> Tuple[Any, 'ImmutableStack']:
        return (self.value, self.tail)

In [3]:
FilterInfo = namedtuple(
    'FilterInfo',
    ('field_name', 'op_name', 'value'),
)
DataToken = TypeVar('DataToken')


class DataContext(Generic[DataToken]):
    
    __slots__ = (
        'current_token',
        'token_at_location',
        'expression_stack',
    )
    
    def __init__(
        self,
        current_token: Optional[DataToken], 
        token_at_location: Dict[Location, Optional[DataToken]], 
        expression_stack: ImmutableStack,
    ):
        self.current_token = current_token
        self.token_at_location = token_at_location
        self.expression_stack = expression_stack
        
    def __repr__(self):
        return 'DataContext(current={}, locations={}, stack={})'.format(
            self.current_token, pformat(self.token_at_location), pformat(self.expression_stack))
        
    __str__ = __repr__
        
    @staticmethod
    def make_empty_context_from_token(token: DataToken) -> 'DataContext':
        return DataContext(token, dict(), make_empty_stack())
    
    def push_value_onto_stack(self, value: Any) -> 'DataContext':
        self.expression_stack = self.expression_stack.push(value)
        return self  # for chaining
    
    def peek_value_on_stack(self) -> Any:
        return self.expression_stack.peek()
        
    def pop_value_from_stack(self) -> Any:
        value, remaining_stack = self.expression_stack.pop()
        self.expression_stack = remaining_stack
        return value
    
    def get_context_for_location(self, location: Location) -> 'DataContext':
        return DataContext(
            self.token_at_location[location], 
            dict(self.token_at_location), 
            self.expression_stack,
        )
        

class InterpreterAdapter(Generic[DataToken], metaclass=ABCMeta):
    @abstractmethod
    def get_tokens_of_type(
        self,
        type_name: str, 
        **hints
    ) -> Iterable[DataToken]:
        pass

    @abstractmethod
    def project_property(
        self,
        data_contexts: Iterable[DataContext], 
        field_name: str,
        **hints
    ) -> Iterable[Tuple[DataContext, Any]]:
        pass

    @abstractmethod
    def project_neighbors(
        self,
        data_contexts: Iterable[DataContext], 
        direction: str,
        edge_name: str, 
        **hints
    ) -> Iterable[Tuple[DataContext, Iterable[DataToken]]]:
        # If using a generator instead of a list for the Iterable[DataToken] part,
        # be careful -- generators are not closures! Make sure any state you pull into
        # the generator from the outside does not change, or that bug will be hard to find.
        # Remember: it's always safer to use a function to produce the generator, since
        # that will explicitly preserve all the external values passed into it.
        pass

    @abstractmethod
    def can_coerce_to_type(
        self,
        data_contexts: Iterable[DataContext], 
        type_name: str,
        **hints
    ) -> Iterable[Tuple[DataContext, bool]]:
        pass

In [4]:
def _apply_operator(operator: str, left_value: Any, right_value: Any) -> Any:
    if operator == '=':
        return left_value == right_value
    elif operator == 'contains':
        return right_value in left_value
    else:
        raise NotImplementedError()

In [5]:
def _push_values_onto_data_context_stack(
    contexts_and_values: Iterable[Tuple[DataContext, Any]]
) -> Iterable[DataContext]:
    return (
        data_context.push_value_onto_stack(value)
        for data_context, value in contexts_and_values
    )


def _evaluate_binary_composition(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    expression: BinaryComposition,
    data_contexts: Iterable[DataContext],
) -> Iterable[Tuple[DataContext, Any]]:
    data_contexts = _push_values_onto_data_context_stack(
        _evaluate_expression(adapter, query_arguments, expression.left, data_contexts)
    )
    data_contexts = _push_values_onto_data_context_stack(
        _evaluate_expression(adapter, query_arguments, expression.right, data_contexts)
    )
    
    for data_context in data_contexts:
        # N.B.: The left sub-expression is evaluated first, therefore its value in the stack
        #       is *below* the value of the right sub-expression.
        #       These two lines cannot be inlined into the _apply_operator() call since
        #       the popping order there would be incorrectly reversed.
        right_value = data_context.pop_value_from_stack()
        left_value = data_context.pop_value_from_stack()
        final_expression_value = _apply_operator(expression.operator, left_value, right_value)
        yield (data_context, final_expression_value)


def _evaluate_ternary_conditional(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    expression: TernaryConditional,
    data_contexts: Iterable[DataContext],
) -> Iterable[Tuple[DataContext, Any]]:
    # TODO(predrag): Try to optimize this to avoid evaluating sides of expressions we might not use.
    data_contexts = _push_values_onto_data_context_stack(
        _evaluate_expression(adapter, query_arguments, expression.predicate, data_contexts)
    )
    data_contexts = _push_values_onto_data_context_stack(
        _evaluate_expression(adapter, query_arguments, expression.if_true, data_contexts)
    )
    data_contexts = _push_values_onto_data_context_stack(
        _evaluate_expression(adapter, query_arguments, expression.if_false, data_contexts)
    )
    
    for data_context in data_contexts:
        # N.B.: The expression evaluation order is "predicate, if_true, if_false", and since the
        #       results are pushed onto a stack (LIFO order), the pop order has to be inverted.
        if_false_value = data_context.pop_value_from_stack()
        if_true_value = data_context.pop_value_from_stack()
        predicate_value = data_context.pop_value_from_stack()
        result_value = if_true_value if predicate_value else if_false_value
        yield (data_context, result_value)

        
def _evaluate_local_field(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    expression: LocalField,
    data_contexts: Iterable[DataContext],
) -> Iterable[Tuple[DataContext, Any]]:
    field_name = expression.field_name
    return adapter.project_property(data_contexts, field_name)


def _evaluate_context_field(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    expression: Union[ContextField, OutputContextField],
    data_contexts: Iterable[DataContext],
) -> Iterable[Tuple[DataContext, Any]]:
    location = expression.location.at_vertex()
    field_name = expression.location.field
    
    moved_contexts = (
        data_context.get_context_for_location(location).push_value_onto_stack(data_context)
        for data_context in data_contexts
    )
    
    return (
        (moved_data_context.pop_value_from_stack(), value)
        for moved_data_context, value in adapter.project_property(moved_contexts, field_name)
    )


def _evaluate_context_field_existence(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    expression: ContextFieldExistence,
    data_contexts: Iterable[DataContext],
) -> Iterable[Tuple[DataContext, Any]]:
    location = expression.location.at_vertex()
    
    for data_context in data_contexts:
        existence_value = data_context.token_at_location[location] is not None
        yield (data_context, existence_value)

    
def _evaluate_variable(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    expression: Variable,
    data_contexts: Iterable[DataContext],
) -> Iterable[Tuple[DataContext, Any]]:
    variable_value = query_arguments[expression.variable_name[1:]]
    return (
        (data_context, variable_value)
        for data_context in data_contexts
    )


def _evaluate_literal(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    expression: Literal,
    data_contexts: Iterable[DataContext],
) -> Iterable[Tuple[DataContext, Any]]:
    return (
        (data_context, expression.value)
        for data_context in data_contexts
    )


def _evaluate_expression(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    expression: Expression,
    data_contexts: Iterable[DataContext],
) -> Iterable[Tuple[DataContext, Any]]:
    type_to_handler = {
        BinaryComposition: _evaluate_binary_composition,
        TernaryConditional: _evaluate_ternary_conditional,
        ContextField: _evaluate_context_field,
        OutputContextField: _evaluate_context_field,
        LocalField: _evaluate_local_field,
        ContextFieldExistence: _evaluate_context_field_existence,
        Variable: _evaluate_variable,
        Literal: _evaluate_literal,
    }
    expression_type = type(expression)
    return type_to_handler[expression_type](adapter, query_arguments, expression, data_contexts)

In [6]:
def _handle_filter(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    block: Filter,
    data_contexts: Iterable[DataContext],
) -> Iterable[DataContext]:
    predicate = block.predicate
    
    # TODO(predrag): Handle the "filters depending on missing optional values pass" rule.
    
    yield from (
        data_context
        for data_context, predicate_value in _evaluate_expression(
            adapter, query_arguments, predicate, data_contexts
        )
        if predicate_value or data_context.current_token is None
    )
    

def _handle_traverse(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: Traverse,
    data_contexts: Iterable[DataContext],
) -> Iterable[DataContext]:
    neighbor_data = adapter.project_neighbors(data_contexts, block.direction, block.edge_name)
    for data_context, neighbor_tokens in neighbor_data:
        has_neighbors = False
        for neighbor_token in neighbor_tokens:
            has_neighbors = True
            yield (
                # TODO(predrag): Make a helper staticmethod on DataContext for this.
                DataContext(
                    neighbor_token, data_context.token_at_location, data_context.expression_stack
                )
            )
        if block.optional and not has_neighbors:
            yield DataContext(
                None, data_context.token_at_location, data_context.expression_stack
            )

In [7]:
def _handle_recurse(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: Recurse,
    data_contexts: Iterable[DataContext],
) -> Iterable[DataContext]:
    data_contexts = _handle_already_inactive_tokens(data_contexts)
    
    for current_depth in range(block.depth):
        data_contexts = _iterative_recurse_handler(
            adapter, query_arguments, block, data_contexts, current_depth
        )
        
    return _unwrap_recursed_data_contexts(data_contexts)
    
    
def _handle_already_inactive_tokens(
    data_contexts: Iterable[DataContext],
) -> Iterable[DataContext]:
    for data_context in data_contexts:
        current_token = data_context.current_token
        if current_token is None:
            # Got a context that is already deactivated at the start of the recursion.
            # Push "None" onto the stack to make sure it remains deactivated at the end of it too.
            data_context.push_value_onto_stack(None)
        yield data_context
    
    
def _iterative_recurse_handler(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: Recurse,
    data_contexts: Iterable[DataContext],
    current_depth: int,
) -> Iterable[DataContext]:
    neighbor_data = adapter.project_neighbors(data_contexts, block.direction, block.edge_name)
    for data_context, neighbor_tokens in neighbor_data:
        yield from (
            # TODO(predrag): Make a helper staticmethod on DataContext for this.
            DataContext(
                neighbor_token, data_context.token_at_location, data_context.expression_stack
            )
            for neighbor_token in neighbor_tokens
        )
        current_token = data_context.current_token
        if current_token is None:
            # The context is already inactive so its neighbors
            # will not be visited in the next iteration.
            yield data_context
        else:
            # We just visited this context's neighbors, deactivate the context
            # so we don't end up visiting them again in the next iteration.
            data_context.push_value_onto_stack(current_token)
            yield DataContext(
                None, data_context.token_at_location, data_context.expression_stack
            )
        

def _unwrap_recursed_data_contexts(
    data_contexts: Iterable[DataContext]
) -> Iterable[DataContext]:
    for data_context in data_contexts:
        if data_context.current_token is not None:
            # Got a still-active context, produce it as-is.
            yield data_context
        else:
            # Got an inactivated context, reactivate it by replacing the token from the stack.
            current_token = data_context.pop_value_from_stack()
            yield DataContext(
                current_token, data_context.token_at_location, data_context.expression_stack
            )

In [8]:
def _produce_output(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    output_name: str,
    output_expression: Expression,
    data_contexts: Iterable[DataContext],
) -> Iterable[DataContext]:
    data_contexts = _print_tap(
        'outputting ' + output_name, data_contexts)
    
    contexts_and_values = _evaluate_expression(
        adapter, query_arguments, output_expression, data_contexts)
    
    for data_context, value in contexts_and_values:
        data_context.peek_value_on_stack()[output_name] = value
        yield data_context
    

def _handle_construct_result(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    block: ConstructResult,
    data_contexts: Iterable[DataContext],
) -> Iterable[Dict[str, Any]]:
    output_fields = block.fields
    
    data_contexts = (
        data_context.push_value_onto_stack(dict())
        for data_context in data_contexts
    )
    
    for output_name, output_expression in output_fields.items():
        data_contexts = _produce_output(
            adapter, query_arguments, output_name, output_expression, data_contexts)
        
    return (
        data_context.pop_value_from_stack()
        for data_context in data_contexts
    )

In [9]:
def _handle_coerce_type(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: CoerceType,
    data_contexts: Iterable[DataContext],
) -> Iterable[DataContext]:
    coercion_type = get_only_element_of_collection(block.target_class)
    return (
        data_context
        for data_context, can_coerce in adapter.can_coerce_to_type(data_contexts, coercion_type)
        if can_coerce or data_context.current_token is None
    )
    

def _handle_mark_location(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: MarkLocation,
    data_contexts: Iterable[DataContext],
) -> Iterable[DataContext]:
    current_location = block.location
    for data_context in data_contexts:
        token_at_location = dict(data_context.token_at_location)
        token_at_location[current_location] = data_context.current_token
        yield DataContext(
            data_context.current_token,
            token_at_location,
            data_context.expression_stack,
        )
        

def _handle_backtrack(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: Backtrack,
    data_contexts: Iterable[DataContext],
) -> Iterable[DataContext]:
    backtrack_location = block.location
    for data_context in data_contexts:
        yield DataContext(
            data_context.token_at_location[backtrack_location],
            data_context.token_at_location,
            data_context.expression_stack,
        )

In [10]:
def _handle_block(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: BasicBlock,
    data_contexts: Iterable[DataContext],
) -> Iterable[DataContext]:
    no_op_types = (EndOptional, GlobalOperationsStart,)
    if isinstance(block, no_op_types):
        return data_contexts
    
    data_contexts = _print_tap('pre: ' + str(block), data_contexts)
    
    handler_functions = {        
        CoerceType: _handle_coerce_type,
        Filter: _handle_filter,
        MarkLocation: _handle_mark_location,
        Traverse: _handle_traverse,
        Recurse: _handle_recurse,
        Backtrack: _handle_backtrack,
    }
    return handler_functions[type(block)](adapter, query_arguments, block, data_contexts)


In [11]:
def _print_tap(info: str, data_contexts: Iterable[DataContext]) -> Iterable[DataContext]:
    return data_contexts
#     print('\n')
#     unique_id = hash(info)
#     print(unique_id, info)
#     from funcy.py3 import chunks
#     for context_chunk in chunks(100, data_contexts):
#         for context in context_chunk:
#             pprint((unique_id, context))
#             yield context
        

In [12]:
def interpret_ir(
    adapter: InterpreterAdapter[DataToken], 
    ir_and_metadata: IrAndMetadata, 
    query_arguments: Dict[str, Any]
) -> Iterable[Dict[str, Any]]:
    ir_blocks = ir_and_metadata.ir_blocks
    query_metadata_table = ir_and_metadata.query_metadata_table
    
    if not ir_blocks:
        raise AssertionError()
        
    first_block = ir_blocks[0]
    if not isinstance(first_block, QueryRoot):
        raise AssertionError()
        
    last_block = ir_blocks[-1]
    if not isinstance(last_block, ConstructResult):
        raise AssertionError()
        
    middle_blocks = ir_blocks[1:-1]
        
    start_class = get_only_element_from_collection(first_block.start_class)
    
    current_data_contexts = (
        DataContext.make_empty_context_from_token(token)
        for token in adapter.get_tokens_of_type(start_class)
    )
    
    current_data_contexts = _print_tap('starting contexts', current_data_contexts)
    
    for block in middle_blocks:
        current_data_contexts = _handle_block(
            adapter, query_arguments, block, current_data_contexts)
        
    current_data_contexts = _print_tap('ending contexts', current_data_contexts)
        
    return _handle_construct_result(
        adapter, query_arguments, last_block, current_data_contexts)
    

In [13]:
vertices = {
    'Animal': [
        {'name': 'Scooby Doo', 'uuid': '1001'},
        {'name': 'Hedwig', 'uuid': '1002'},
        {'name': 'Beethoven', 'uuid': '1003'},
        {'name': 'Pongo', 'uuid': '1004'},
        {'name': 'Perdy', 'uuid': '1005'},
        {'name': 'Dipstick', 'uuid': '1006'},
        {'name': 'Dottie', 'uuid': '1007'},
        {'name': 'Domino', 'uuid': '1008'},
        {'name': 'Little Dipper', 'uuid': '1009'},
        {'name': 'Oddball', 'uuid': '1010'},
    ],
}
edges = {
    'Animal_ParentOf': [
        ('1004', '1006'),
        ('1005', '1006'),
        ('1006', '1008'),
        ('1006', '1009'),
        ('1006', '1010'),
        ('1007', '1008'),
        ('1007', '1009'),
        ('1007', '1010'),
    ],
}

vertices_by_uuid = {
    vertex['uuid']: vertex
    for vertex in chain.from_iterable(vertices.values())
}


class InMemoryAdapter(InterpreterAdapter[dict]):
    def get_tokens_of_type(
        self,
        type_name: str, 
        **hints
    ) -> Iterable[dict]:
        return vertices[type_name]

    def project_property(
        self,
        data_contexts: Iterable[DataContext], 
        field_name: str,
        **hints
    ) -> Iterable[Tuple[DataContext, Any]]:
        for data_context in data_contexts:
            current_token = data_context.current_token
            current_value = current_token[field_name] if current_token is not None else None
            yield (data_context, current_value)

    def project_neighbors(
        self,
        data_contexts: Iterable[DataContext], 
        direction: str,
        edge_name: str, 
        **hints
    ) -> Iterable[Tuple[DataContext, Iterable[DataToken]]]:
        edge_info = edges[edge_name]
        
        for data_context in data_contexts:
            neighbor_tokens = []
            current_token = data_context.current_token
            if current_token is not None:
                uuid = current_token['uuid']
                if direction == 'out':
                    neighbor_tokens = [
                        vertices_by_uuid[destination_uuid]
                        for source_uuid, destination_uuid in edge_info
                        if source_uuid == uuid
                    ]
                elif direction == 'in':
                    neighbor_tokens = [
                        vertices_by_uuid[destination_uuid]
                        for source_uuid, destination_uuid in edge_info
                        if destination_uuid == uuid
                    ]
                else:
                    raise AssertionError()
                
            yield (data_context, neighbor_tokens)

    def can_coerce_to_type(
        self,
        data_contexts: Iterable[DataContext], 
        type_name: str,
        **hints
    ) -> Iterable[Tuple[DataContext, bool]]:
        # TODO(predrag): See if a redesign can make this be a no-op again.
        return zip(data_contexts, repeat(True))

In [14]:
schema = build_ast_schema(parse(SCHEMA_TEXT))

In [15]:
query = '''
{
    Animal {
        name @output(out_name: "animal_name")
        uuid @output(out_name: "animal_uuid")
    }
}
'''
query_arguments = {}

In [16]:
query = '''
{
    Animal {
        name @output(out_name: "parent_name")

        out_Animal_ParentOf {
            name @output(out_name: "child_name")
        }
    }
}
'''
query_arguments = {}

In [17]:
query = '''{
    Animal {
        name @output(out_name: "parent_name")
        
        out_Animal_ParentOf @optional {
            name @filter(op_name: "in_collection", value: ["$child_names"])
                 @output(out_name: "child_name")
        }
    }
}'''
query_arguments = {
    "child_names": ['Domino', 'Dipstick', 'Oddball'],
}

In [18]:
query = '''{
    Animal {
        name @output(out_name: "ancestor_or_self_name")
        
        out_Animal_ParentOf @recurse(depth: 4) {
            name @filter(op_name: "in_collection", value: ["$names"])
                 @output(out_name: "descendant_or_self_name")
        }
    }
}'''
query_arguments = {
    "names": ['Domino', 'Dipstick', 'Oddball'],
}

In [19]:
query = '''{
    Animal {
        name @output(out_name: "ancestor_or_self_name")
        
        out_Animal_ParentOf @recurse(depth: 1) {
            name @filter(op_name: "in_collection", value: ["$names"])
                 @output(out_name: "descendant_or_self_name")
        }
    }
}'''
query_arguments = {
    "names": ['Domino', 'Dipstick', 'Oddball'],
}

In [20]:
query = '''
{
    Animal {
        name @output(out_name: "parent_name")

        out_Animal_ParentOf {
            name @filter(op_name: "in_collection", value: ["$child_names"])
                 @output(out_name: "child_name")
        }
    }
}
'''
query_arguments = {
    "child_names": ['Domino', 'Dipstick', 'Oddball'],
}

In [21]:
query = '''
{
    Animal {
        name @output(out_name: "grandparent_name")

        out_Animal_ParentOf {
            name @output(out_name: "parent_name")
            
            out_Animal_ParentOf {
                name @output(out_name: "child_name")
            }
        }
    }
}
'''
query_arguments = {}

In [22]:
ir_and_metadata = graphql_to_ir(schema, query)
pprint(ir_and_metadata.ir_blocks)

result = list(interpret_ir(InMemoryAdapter(), ir_and_metadata, query_arguments))
result

[QueryRoot(({'Animal'},)),
 MarkLocation((Location(('Animal',), None, 1),)),
 Traverse(('out', 'Animal_ParentOf'), {'optional': False, 'within_optional_scope': False}),
 MarkLocation((Location(('Animal', 'out_Animal_ParentOf'), None, 1),)),
 Traverse(('out', 'Animal_ParentOf'), {'optional': False, 'within_optional_scope': False}),
 MarkLocation((Location(('Animal', 'out_Animal_ParentOf', 'out_Animal_ParentOf'), None, 1),)),
 Backtrack((Location(('Animal', 'out_Animal_ParentOf'), None, 1),), {'optional': False}),
 Backtrack((Location(('Animal',), None, 1),), {'optional': False}),
 GlobalOperationsStart(),
 ConstructResult(({'grandparent_name': OutputContextField((Location(('Animal',), name, 1), <graphql.type.definition.GraphQLScalarType object at 0x10659fb40>)), 'parent_name': OutputContextField((Location(('Animal', 'out_Animal_ParentOf'), name, 1), <graphql.type.definition.GraphQLScalarType object at 0x10659fb40>)), 'child_name': OutputContextField((Location(('Animal', 'out_Animal_Pare

[{'grandparent_name': 'Pongo',
  'parent_name': 'Dipstick',
  'child_name': 'Domino'},
 {'grandparent_name': 'Pongo',
  'parent_name': 'Dipstick',
  'child_name': 'Little Dipper'},
 {'grandparent_name': 'Pongo',
  'parent_name': 'Dipstick',
  'child_name': 'Oddball'},
 {'grandparent_name': 'Perdy',
  'parent_name': 'Dipstick',
  'child_name': 'Domino'},
 {'grandparent_name': 'Perdy',
  'parent_name': 'Dipstick',
  'child_name': 'Little Dipper'},
 {'grandparent_name': 'Perdy',
  'parent_name': 'Dipstick',
  'child_name': 'Oddball'}]